In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-07-18 12:38:48--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1,1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1,06M  6,55MB/s    in 0,2s    

2024-07-18 12:38:49 (6,55 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [2]:
with open('input.txt', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of text: ", len(text))

length of text:  1115394


In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
chars = sorted(set(text))
vocab_size = len(chars)
print(''.join(chars))
print('Vocab size: ', vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vocab size:  65


In [6]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda x: [stoi[ch] for ch in x]
decode = lambda x: ''.join([itos[i] for i in x])

print(encode('hii there'))
print(decode(encode('hii there')))


[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [8]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [9]:
n = int(0.9 * len(data))
train_data, val_data = data[:n], data[n:]

In [10]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for t in range(block_size):
    context = x[:t + 1]
    target = y[t]
    print(f"when input is {context} then target is {target}")

when input is tensor([18]) then target is 47
when input is tensor([18, 47]) then target is 56
when input is tensor([18, 47, 56]) then target is 57
when input is tensor([18, 47, 56, 57]) then target is 58
when input is tensor([18, 47, 56, 57, 58]) then target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) then target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) then target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) then target is 58


In [58]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(data.size(0) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + 1 + block_size] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs: ', xb.shape, xb.dtype)
print(xb)
print('targets: ', yb.shape, yb.dtype)
print(yb)

print('----')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t + 1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} then target is {target}")


inputs:  torch.Size([4, 8]) torch.int64
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:  torch.Size([4, 8]) torch.int64
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] then target is 43
when input is [24, 43] then target is 58
when input is [24, 43, 58] then target is 5
when input is [24, 43, 58, 5] then target is 57
when input is [24, 43, 58, 5, 57] then target is 1
when input is [24, 43, 58, 5, 57, 1] then target is 46
when input is [24, 43, 58, 5, 57, 1, 46] then target is 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] then target is 39
when input is [44] then target is 53
when input is [44, 53] then target is 56
when input is [44, 53, 56] then target is 1
when input is [44, 53, 56, 1] then target is 58
when inp

In [52]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, idx_next], dim=1)
        return idx

    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long)
gen_seq = m.generate(idx, max_new_tokens=100)
print(decode(gen_seq[0].tolist()))


torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [57]:
optimizer = torch.optim.Adam(m.parameters(), lr=0.001)

In [106]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if steps % 10 == 0:
        print(f'loss: {loss.item()}')

loss: 2.616670846939087
loss: 2.642460823059082
loss: 2.4217329025268555
loss: 2.6373305320739746
loss: 2.5708348751068115
loss: 2.5370733737945557
loss: 2.6140685081481934
loss: 2.539964199066162
loss: 2.7259726524353027
loss: 2.5714969635009766
loss: 2.4518954753875732
loss: 2.671290874481201
loss: 2.590604305267334
loss: 2.5630486011505127
loss: 2.5906753540039062
loss: 2.6879000663757324
loss: 2.5954761505126953
loss: 2.535547971725464
loss: 2.5478150844573975
loss: 2.520815372467041
loss: 2.422400712966919
loss: 2.49772572517395
loss: 2.495818853378296
loss: 2.459104299545288
loss: 2.4867002964019775
loss: 2.553175926208496
loss: 2.530959129333496
loss: 2.6377639770507812
loss: 2.493283748626709
loss: 2.500706672668457
loss: 2.620847225189209
loss: 2.5842771530151367
loss: 2.5822267532348633
loss: 2.5345070362091064
loss: 2.507331132888794
loss: 2.698512315750122
loss: 2.414506673812866
loss: 2.4676430225372314
loss: 2.4631714820861816
loss: 2.54647159576416
loss: 2.51814723014831

In [108]:
idx = torch.zeros((1, 1), dtype=torch.long)
gen_seq = m.generate(idx, max_new_tokens=1000)
print(decode(gen_seq[0].tolist()))


Fofur af II dwid ty yot thevaustome f preay ld, s t nchanthinf an k healoblldond hininche th ld is.

Thye hatst thus
Ou? mespriowh codyove d.
I nete theiapoba wheinsothe Cord sithine p lfulther, wallofe t ICo:
ME:
OUMERLO:
NI wayouroikl y ftwaveat he'
St H:
Lurthou hefamy ipont y te y py wide. th ce br mistorasthame Oust orinkeal qureribot is ddy nechthon m t; ghen than.
HMI al ind mprs thie!
Wed rur,-d, therothanowesll g t betert d?
ANonjunfu e ting nd, whefacowoke cowh ds,
O:
Fo che te, IUSepeliformarrorank mincourecharenve awh s,
GLAnct, INams wimuthar
WAUSS:
Bur tis ey ou rssensoshorswin wesse nd l fe oninor y macinee therouneme atho or wise indo, t rn asllianeedend ctr me?
wheint t p, ch lindisearouis tl'so I therowile hingrthad
ORULLLUTI a mato;
WBRino beay IUS:
Whowofrd s athatsond ousthiflad d oul t f ime, am he: iche

AMos, k wherman de fam ban y o ing IDWhend mey, k,
ORICo ou fuswais theforobjusmen, ma,
WAlas
Datrncrayord.
TRD orme!
Wherinthal d cris,
Buss b
Whoy I r n d she